# 2.3 Experiment tracking with MLflow

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

 ### ML Flow

In [5]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

# access UI
# mlflow ui --backend-store-uri sqlite:///mlflow.db

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1732646263960, experiment_id='1', last_update_time=1732646263960, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
# create utility function for dataframe reading
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

### Import datasets

In [7]:
# january
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet')
# february
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet')
len(df_train), len(df_val)

(54373, 51497)

In [8]:
# combine Pick Up Location and Drop Off Location in one feature
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()
# fit transform
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
# transform
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

X_train, X_val

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 108746 stored elements and shape (54373, 5093)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 100591 stored elements and shape (51497, 5093)>)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

### Train models

In [9]:
# fit Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
# predict on validation
y_val_hat = lr.predict(X_val)
root_mean_squared_error(y_val, y_val_hat)

np.float64(5.995041447869019)

In [10]:
# fit Lasso Regression

# ML Flow log
with mlflow.start_run():

    # run tag: utils for run filtering
    mlflow.set_tag('developer', 'leonardo')

    # useful parameters
    mlflow.log_param('train-data-path', 'green_tripdata_2024-01.parquet')
    mlflow.log_param('val-data-path', 'green_tripdata_2024-02.parquet')
    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)
    # predict on validation
    y_val_hat = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_val_hat)

    # metrics
    mlflow.log_metric('rmse', rmse)

In [11]:
# fit Ridge Regression
lr = Ridge()
lr.fit(X_train, y_train)
# predict on validation
y_val_hat = lr.predict(X_val)
root_mean_squared_error(y_val, y_val_hat)

np.float64(5.980948972923878)

In [12]:
# save model
with open('models/taxi_ridge_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

### Hyperparameters search with Hyperopt and MLflow

In [13]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [15]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric('rmse', rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [16]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),  # [exp(-3), exp(0)] = [-0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

In [17]:
# fmin method tries to minimize the obj function with the algorithm tpe
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)

[0]	validation-rmse:8.76272                                                                                                                          
[1]	validation-rmse:8.44136                                                                                                                          
[2]	validation-rmse:8.14580                                                                                                                          
[3]	validation-rmse:7.87391                                                                                                                          
[4]	validation-rmse:7.62480                                                                                                                          
[5]	validation-rmse:7.39631                                                                                                                          
[6]	validation-rmse:7.18766                                                                         

### Train best model

In [18]:
# autolog works only with certain frameworks -> check documentation
best_params = {
    'learning_rate': 0.7796116539991661,
    'max_depth': 58,
    'min_child_weight': 5.171721905211863,
    'objective': 'reg:squarederror',
    'reg_alpha': 0.3147724697905352,
    'reg_lambda': 0.01549469142014421,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2024/11/27 17:59:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a9a9806159914359bc8dabfa5790095c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:5.60909
[1]	validation-rmse:5.34361
[2]	validation-rmse:5.30665
[3]	validation-rmse:5.29071
[4]	validation-rmse:5.28842
[5]	validation-rmse:5.29025
[6]	validation-rmse:5.27345
[7]	validation-rmse:5.26109
[8]	validation-rmse:5.25938
[9]	validation-rmse:5.24434
[10]	validation-rmse:5.24280
[11]	validation-rmse:5.24408
[12]	validation-rmse:5.24225
[13]	validation-rmse:5.24407
[14]	validation-rmse:5.24436
[15]	validation-rmse:5.24270
[16]	validation-rmse:5.23825
[17]	validation-rmse:5.23727
[18]	validation-rmse:5.23499
[19]	validation-rmse:5.23482
[20]	validation-rmse:5.23417
[21]	validation-rmse:5.23275
[22]	validation-rmse:5.23263
[23]	validation-rmse:5.22892
[24]	validation-rmse:5.23092
[25]	validation-rmse:5.22924
[26]	validation-rmse:5.22964
[27]	validation-rmse:5.22905
[28]	validation-rmse:5.22843
[29]	validation-rmse:5.22721
[30]	validation-rmse:5.22918
[31]	validation-rmse:5.22771
[32]	validation-rmse:5.23057
[33]	validation-rmse:5.22846
[34]	validation-rmse:5.2

2024/11/27 17:59:42 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/11/27 17:59:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:59:42] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/11/27 17:59:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


---
# 2.4 Model management

### Save model artifacts

#### 1. Naive

In [21]:
# fit Lasso Regression

# ML Flow log
with mlflow.start_run():

    # run tag: utils for run filtering
    mlflow.set_tag('developer', 'leonardo')
    mlflow.set_tag('model', 'lasso')

    # useful parameters
    mlflow.log_param('train-data-path', 'green_tripdata_2024-01.parquet')
    mlflow.log_param('val-data-path', 'green_tripdata_2024-02.parquet')
    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)
    # predict on validation
    y_val_hat = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_val_hat)

    # metrics
    mlflow.log_metric('rmse', rmse)

    with open('models/lasso_reg.bin', 'wb') as f_out:
        pickle.dump((dv, lr), f_out)

    mlflow.log_artifact(local_path='models/lasso_reg.bin', artifact_path='models_pickle/lasso_reg.bin')

#### 2. Advanced 

In [25]:
import xgboost as xgb

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

mlflow.xgboost.autolog(disable=True)

with mlflow.start_run():
    best_params = {
        'learning_rate': 0.7796116539991661,
        'max_depth': 58,
        'min_child_weight': 5.171721905211863,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.3147724697905352,
        'reg_lambda': 0.01549469142014421,
        'seed': 42
    }
    
    mlflow.log_params(best_params)
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    with open ('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
        
    # save preprocessing as artifact
    mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

    # model saving
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

[0]	validation-rmse:5.60909
[1]	validation-rmse:5.34361
[2]	validation-rmse:5.30665
[3]	validation-rmse:5.29071
[4]	validation-rmse:5.28842
[5]	validation-rmse:5.29025
[6]	validation-rmse:5.27345
[7]	validation-rmse:5.26109
[8]	validation-rmse:5.25938
[9]	validation-rmse:5.24434
[10]	validation-rmse:5.24280
[11]	validation-rmse:5.24408
[12]	validation-rmse:5.24225
[13]	validation-rmse:5.24407
[14]	validation-rmse:5.24436
[15]	validation-rmse:5.24270
[16]	validation-rmse:5.23825
[17]	validation-rmse:5.23727
[18]	validation-rmse:5.23499
[19]	validation-rmse:5.23482
[20]	validation-rmse:5.23417
[21]	validation-rmse:5.23275
[22]	validation-rmse:5.23263
[23]	validation-rmse:5.22892
[24]	validation-rmse:5.23092
[25]	validation-rmse:5.22924
[26]	validation-rmse:5.22964
[27]	validation-rmse:5.22905
[28]	validation-rmse:5.22843
[29]	validation-rmse:5.22721
[30]	validation-rmse:5.22918
[31]	validation-rmse:5.22771
[32]	validation-rmse:5.23057
[33]	validation-rmse:5.22846
[34]	validation-rmse:5.2

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:56:43] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/12/04 17:56:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Import Artifact

#### 1. Log as python function

In [28]:
logged_model = 'runs:/4ae919aba7d04337a76c12fc214fdd31/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 4ae919aba7d04337a76c12fc214fdd31

#### 2. Log as mlflow model

In [29]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

### Predict

In [30]:
xgboost_model.predict(valid)

array([19.275356, 28.798018,  8.098671, ..., 32.05554 , 13.361015,
       20.312605], dtype=float32)

---
# 2.5 Model registry